# Генерация белковых молекул

! pip install --upgrade git+https://github.com/rimgro/biocadprotein.git

In [1]:
from fpgen.generation.generator import ProteinGenerator
from fpgen.generation.metrics import Metric
from fpgen.utils import get_active_site_residues
from fpgen.visualization import (
    plot_3d_protein,
    plot_2d_protein
)

from esm.models.esm3 import ESM3
from esm.sdk.api import ESMProtein
from esm.utils.structure.protein_chain import ProteinChain

In [2]:
# Создание модели ESM3 для генерации молекул
model = ESM3.from_pretrained('esm3-open').to('cuda')

Fetching 22 files:   0%|          | 0/22 [00:00<?, ?it/s]

In [3]:
gfp_protein = ESMProtein.from_protein_chain(
    ProteinChain.from_rcsb('1qy3', chain_id='A')
)

idx_alpha_phelix = get_active_site_residues(
    (56, 76),
    protein=gfp_protein,
    radius=5
)

c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\esm\utils\structure\protein_complex.py:223: UserWarning: Entity ID not found in metadata, using None as default
  warnings.warn("Entity ID not found in metadata, using None as default")


In [4]:
generator = ProteinGenerator(gfp_protein, idx_alpha_phelix, model)

In [6]:
generated_protein, metric = generator.generate(
    # (calculate_on_full_atom означает, что мы будем расчитывать метрику на полноатомной структуре),
    # а не на бэкбоне
    metric_list=[
        Metric('rmsd', calculate_on_full_atom=True),
        Metric('ptm'),
        Metric('plddt')
    ],
    # Сгенерировать также полноатомную структуру белка
    fix_protein=True
)

100%|██████████| 20/20 [00:14<00:00,  1.37it/s]
c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\esm\utils\structure\protein_complex.py:223: UserWarning: Entity ID not found in metadata, using None as default
  warnings.warn("Entity ID not found in metadata, using None as default")


In [7]:
metric_names = ['RMSD', 'pTM', 'pLDDT']

for metric_name, metric_val in zip(metric_names, metric):
    print(f'{metric_name}: {metric_val}')

RMSD: 2.081880391305786
pTM: 0.8098708987236023
pLDDT: 0.8426768183708191


Теперь можно получить:

- ```generated_protein['backbone']```: бэкбон молекулы
- ```generated_protein['full_atom']```: полноатомную структуру белка

In [ ]:
# 3D визуализация белка
plot_3d_protein(generated_protein['full_atom'])